In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# install dependencies: 
!pip install pyyaml==5.1
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
import torch
assert torch.__version__.startswith("1.7")
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu102/torch1.7/index.html

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()


# import some common libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import  StratifiedShuffleSplit
import os, json, cv2, random

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog 

import copy
import logging
import numpy as np
from typing import Callable, List, Union
import torch

from detectron2.config import configurable
from detectron2.data import MetadataCatalog
from detectron2.data import detection_utils as utils
from detectron2.data import transforms as T

from detectron2.data import detection_utils as utils
import copy
import detectron2.data.transforms as T
import matplotlib.pyplot as plt
from detectron2.data import DatasetMapper
import torch
import os
import numpy as np

from detectron2.config import configurable

In [ ]:
dirpath = '../input/vinbigdata-competition-jpg-data-3x-downsampled'

df = pd.read_csv(f'{dirpath}/train_downsampled.csv')
df.shape

In [ ]:
df.head()

In [ ]:
df['w'], df['h'] = df['x_max'] - df['x_min'], df['y_max'] - df['y_min']
df['area'] = df['w'] * df['h']
df.head()

In [ ]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
df_dd = df.drop_duplicates('image_id')
df_dd = df_dd.reset_index()
sss.get_n_splits(df_dd['image_id'], df_dd['class_id'])
for train_index, test_index in sss.split(df_dd['image_id'], df_dd['class_id']):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = df_dd['image_id'][train_index], df_dd['image_id'][test_index]
    y_train, y_test = df_dd['class_id'][train_index], df_dd['class_id'][test_index]

In [ ]:
classes = df.drop_duplicates('class_id').sort_values('class_id')[['class_name']].values[:-1].ravel().tolist()
print(classes)
thing_classes = {class_name: index for index, class_name in enumerate(classes)}
print(thing_classes)

## Read in dicom files

In [ ]:
##### DATASET PREPARING: CONERTING CSV TO DICTIONARY WHICH CONTAINS ANNOTATION AND IMAGE PATH ETC.

from detectron2.structures import BoxMode

def chest_dicts(images, img_dir = '../input/vinbigdata-competition-jpg-data-3x-downsampled/train/train'):
    
    dataset_dicts = []
    for idx, v in enumerate(images):
        record = {}
        
        filename = os.path.join(img_dir, v + '.jpg')
        
        image = cv2.imread(filename)
        height, width, ch = image.shape
        
        record["file_name"] = filename
        record["image_id"] = idx
        record["height"] = height # RANDOM Not Req
        record["width"] = width # RANDOM Not Req
      
        annos = df[df.image_id == v]
        objs = []
        for _, anno in annos.iterrows():
            if anno.class_id != 14:

                obj = {
                    "bbox": [int(anno.x_min), int(anno.y_min), int(anno.w), int(anno.h)],
                    "bbox_mode": BoxMode.XYWH_ABS,
                    "category_id": int(anno.class_id)
                }
                objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts
def train():
    return chest_dicts(X_train)
def val():
    return chest_dicts(X_test)
DatasetCatalog.register("chest_Train", train)
MetadataCatalog.get("chest_Train").set(thing_classes=classes)
DatasetCatalog.register("chest_Val",val)
MetadataCatalog.get("chest_Val").set(thing_classes=classes)
Chest_metadata = MetadataCatalog.get("chest_Train")

In [ ]:
from detectron2.data import detection_utils as utils
from detectron2.data import build_detection_train_loader
from PIL import Image
# writing a new DatasetMapper as we have dicom files

class DatasetMapper:

    @configurable
    def __init__(
        self,
        is_train: bool,
        *,
        augmentations: List[Union[T.Augmentation, T.Transform]],
        image_format: str = 'BGR',
        use_instance_mask: bool = False,
        use_keypoint: bool = False,
        instance_mask_format = "polygon",
        keypoint_hflip_indices = None,
        precomputed_proposal_topk= None,
        recompute_boxes: bool = False,
    ):
        """
        NOTE: this interface is experimental.

        Args:
            is_train: whether it's used in training or inference
            augmentations: a list of augmentations or deterministic transforms to apply
            image_format: an image format supported by :func:`detection_utils.read_image`.
            use_instance_mask: whether to process instance segmentation annotations, if available
            use_keypoint: whether to process keypoint annotations if available
            instance_mask_format: one of "polygon" or "bitmask". Process instance segmentation
                masks into this format.
            keypoint_hflip_indices: see :func:`detection_utils.create_keypoint_hflip_indices`
            precomputed_proposal_topk: if given, will load pre-computed
                proposals from dataset_dict and keep the top k proposals for each image.
            recompute_boxes: whether to overwrite bounding box annotations
                by computing tight bounding boxes from instance mask annotations.
        """

        # fmt: off
        self.is_train               = is_train
        self.augmentations          = T.AugmentationList(  augmentations)
        self.image_format           = image_format
        
        # fmt: on
        logger = logging.getLogger(__name__)
        mode = "training" if is_train else "inference"
        logger.info(f"[DatasetMapper] Augmentations used in {mode}: {augmentations}")

    @classmethod
    def from_config(cls, cfg, is_train: bool = True):
        augs = utils.build_augmentation(cfg, is_train)
        if cfg.INPUT.CROP.ENABLED and is_train:
            augs.insert(0, T.RandomCrop(cfg.INPUT.CROP.TYPE, cfg.INPUT.CROP.SIZE))
            recompute_boxes = cfg.MODEL.MASK_ON
        else:
            recompute_boxes = False


        ret = {
            "is_train": is_train,
            "augmentations": augs,
            "image_format": cfg.INPUT.FORMAT,
            "use_instance_mask": cfg.MODEL.MASK_ON,
            "instance_mask_format": cfg.INPUT.MASK_FORMAT,
            "use_keypoint": cfg.MODEL.KEYPOINT_ON,
            "recompute_boxes": recompute_boxes,
        }

        return ret

    def __call__(self, dataset_dict):
        # print(dataset_dict)
        dataset_dict = copy.deepcopy(dataset_dict)
        image = utils.read_image(dataset_dict["file_name"])
        # I'm not sure about these augmentations, need to properly go over them
        auginput = T.AugInput(image)
        transform = self.augmentations(auginput)
        image = np.expand_dims(auginput.image, axis=2).copy()
        image = torch.from_numpy(image.transpose(2, 0, 1))
        annos = [
            utils.transform_instance_annotations(annotation, [transform], image.shape[1:])
            for annotation in dataset_dict.pop("annotations")
        ]
        return {
        # create the format that the model expects
        "image": image,
        "instances": utils.annotations_to_instances(annos, image.shape[1:])
        }

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.data import MetadataCatalog, build_detection_train_loader,build_detection_test_loader
from detectron2.evaluation import COCOEvaluator

In [ ]:
class Trainer(DefaultTrainer):
    
    @classmethod
    def build_test_loader(cls, cfg, dataset_name):
        return build_detection_test_loader(cfg,dataset_name ,
                mapper=DatasetMapper(cfg, is_train = True))

    @classmethod
    def build_train_loader(cls, cfg):
        return build_detection_train_loader(dataset =train(),
                mapper=DatasetMapper(cfg, is_train = True),
                aspect_ratio_grouping=False, 
                total_batch_size = cfg.SOLVER.IMS_PER_BATCH)
    

In [ ]:
Batch = 10
Epochs = 3
steps = 1000  #  ### INCREASE THE STEPS   (len(X_train) // Batch) * Epochs 
cfg = get_cfg()
NAME = "COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"
cfg.merge_from_file(model_zoo.get_config_file(NAME))
cfg.DATASETS.TRAIN = ("chest_Train",)
cfg.DATASETS.TEST = ('chest_Val', )
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS =  model_zoo.get_checkpoint_url(NAME)
cfg.SOLVER.IMS_PER_BATCH = Batch
cfg.CUDNN_BENCHMARK =  True
cfg.MODEL.RETINANET.NUM_CLASSES  = len(classes)
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.LR_SCHEDULER_NAME = "WarmupCosineLR"
cfg.SOLVER.MAX_ITER = steps  
cfg.OUTPUT_DIR = './output'
cfg.MODEL.PIXEL_MEAN = [103.530]
cfg.MODEL.PIXEL_STD = [1.0]
cfg.SOLVER.CHECKPOINT_PERIOD = 1000
cfg.SOLVER.CLIP_GRADIENTS.CLIP_VALUE = 0.95
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

In [ ]:
trainer = Trainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()